In [152]:
import pickle
import json
import mmcv
from tqdm import tqdm_notebook as tqdm
from mmdet.datasets import build_dataloader, get_dataset
from pycocotools.coco import COCO
import pycocotools.coco as cocoapi
import pycocotools.mask as MASK
import os
import cv2
from icecream import ic
import matplotlib.pyplot as plt
from mmdet.models import build_detector
from mmcv.runner import load_checkpoint, get_dist_info
import numpy as np
def get_original_size(name):
    datadir = "/home/xfr/rssid/data/val/images/"
    files = os.listdir(datadir)
    img_name = ""
    for file in files:
        img_name = file.split(".")[0]
        if(img_name == name):
            break
    filename = os.path.join(datadir, file)
    img = cv2.imread(filename)
    h, w, _ = img.shape
    return h,w
CLASSES = ('tennis-court', 'container-crane', 'storage-tank', 'baseball-diamond', 'plane','ground-track-field',
               'helicopter', 'airport','harbor', 'ship', 'large-vehicle', 'swimming-pool', 'soccer-ball-field',
               'roundabout', 'basketball-court', 'bridge', 'small-vehicle', 'helipad')
cfg = mmcv.Config.fromfile("configs/rs_cascade_mask_rcnn_r50_fpn_1x.py")
results = mmcv.load("new_result.pkl")
dataset = get_dataset(cfg.data.test)
img_infos = dataset.load_annotations("/home/xfr/mmdetection/data/rscup/annotation/annos_rscup_val.json")
ann = {}
last_name = ""
last = ""
for i in tqdm(range(len(results))):
    filename = img_infos[i]['filename']
    img = cv2.imread("./data/rscup/val/" + filename)
    h, w, _ = img.shape
    items = filename.split("_")
    name = items[0]
    if(name == last_name):
        H, W = last
    else:
        H, W = get_original_size(name)
    last_name = name
    last = (H, W)
    if name not in ann:
        ann[name] = {"bbox": [[] for i in range(len(dataset.CLASSES))], "seg": [[] for i in range(len(dataset.CLASSES))],
                     "vis": [[] for i in range(len(dataset.CLASSES))]}
#         ann[name]["mask"] = np.zeros((H,W))
    scale_factor = 1/float(items[1])
    locx = int(float(items[2]))
    locy = int(float(items[3]))
    result = results[i]
    bboxes = result[0]
    segs = result[1]
    for cls in range(len(dataset.CLASSES)):
        bbox_cls = bboxes[cls]
        seg_cls = segs[cls]
        if( len(bbox_cls) > 0):
            for bbox, rle in zip(bbox_cls, seg_cls):
                xmin, ymin, xmax, ymax, score = bbox
#                 cv2.rectangle(img, (xmin, ymin), (xmax, ymax), (0,0,255), 2)
#                 cv2.putText(img, str(cls), (int(xmin), int(ymin)), cv2.FONT_HERSHEY_COMPLEX, 1, (0, 0, 255), 2)
                xmin += locx
                ymin += locy
                xmax += locx
                ymax += locy
                bounding_box = np.array([xmin*scale_factor, ymin*scale_factor, xmax*scale_factor, ymax*scale_factor, score])
                mask = MASK.decode(rle)
#                 print(mask.shape)
#                 mh, mw = mask.shape 
#                 new_mask = cv2.resize(mask, (int(mw*scale_factor),int(mh*scale_factor)))
                
#                 h2, w2 = ann[name]["mask"][int(locy*scale_factor):int(locy*scale_factor)+new_mask.shape[0],int(locx*scale_factor):int(locx*scale_factor)+new_mask.shape[1]].shape
#                 new_mask = cv2.resize(mask, (w2,h2))
#                 ann[name]["mask"][int(locy*scale_factor):int(locy*scale_factor)+new_mask.shape[0],int(locx*scale_factor):int(locx*scale_factor)+new_mask.shape[1]] += new_mask 
                
                contours, hierarchy = cv2.findContours((mask).astype(np.uint8), cv2.RETR_TREE,
                                                                 cv2.CHAIN_APPROX_SIMPLE)
                areas = [cv2.contourArea(contour)  for contour in contours]
                if(len(areas) > 0):
                    index = np.argmax(areas)
                    contour = contours[index]
                    rect = cv2.minAreaRect(contour)
                    box = cv2.boxPoints(rect)
                    box += np.array([locx, locy])
#                     temp = np.array([box], np.int32)
#                     cv2.polylines(img, temp, 1, (0, 255, 0), 1)
#                     cv2.fillPoly(img, temp, 255)
                else:
#                     box = np.array([[xmin, ymin], [xmin, ymax], [xmax, ymax], [xmax, ymin]])
#                     rect = ((0.5*(xmin+xmax), 0.5*(ymin+ymax))),(ymax-ymin, xmax -xmin), 0)
                    continue
                ann[name]["bbox"][cls].append(bounding_box)
                ann[name]['vis'][cls].append(np.array([box[0]*scale_factor, box[1]*scale_factor, box[2]*scale_factor, box[3]*scale_factor]))
                poly = []
                for point in box:
                    poly.append(point[0])
                    poly.append(point[1])
                poly = np.array(poly, np.double)
                poly = [poly*scale_factor]
                rles = MASK.frPyObjects(poly, H, W)
                rle = MASK.merge(rles)
                ann[name]["seg"][cls].append(rle)
    #cv2.imwrite("./vis/"+filename, img)
print(len(ann.keys()))
    #coco_eval(result_file, eval_types, dataset.coco)
    # nms
    # toBbox
    # box - mask - rle
    # rles = maskUtils.frPyObjects(segm, h, w)
       #     rle = maskUtils.merge(rles)
   

loading annotations into memory...
Done (t=0.18s)
creating index...
index created!
loading annotations into memory...
Done (t=2.83s)
creating index...
index created!


593


In [16]:
mmcv.dump(ann, "eval_temp.pkl")

In [58]:
from shapely.geometry import *
import copy
import time
def py_cpu_pnms(dets, pts, thresh):
    dets = np.array(dets)
    pts = np.array(pts)
    scores = dets[:, 4]
    order = scores.argsort()[::-1]
    areas = np.zeros(scores.shape)
    inter_areas = np.zeros((scores.shape[0], scores.shape[0]))
    for il in range(len(pts)):
        poly = Polygon(pts[il])
        areas[il] = poly.area
        for jl in range(il, len(pts)):
            polyj = Polygon(pts[jl])
            inS = poly.intersection(polyj)
            inter_areas[il][jl] = inS.area
            inter_areas[jl][il] = inS.area

    keep = []
    while order.size > 0:
        i = order[0]
        keep.append(i)
        ovr = inter_areas[i][order[1:]] / (areas[i] + areas[order[1:]] - inter_areas[i][order[1:]])
        inds = np.where(ovr <= thresh)[0]
        order = order[inds + 1]
    return np.array(keep, np.int32)
def nms(ann, thresh):
    
    for name in tqdm(ann.keys()):
        info = ann[name]
        for cls in range(len(dataset.CLASSES)):
            bbox = np.array(info['bbox'][cls])
           
            seg = np.array(info['seg'][cls])
            vis = np.array(info['vis'][cls])
            if(len(bbox)<=0):
                continue
            t1 = time.time()
            h, w = seg[0]['size']
            bbox_result = []
            seg_result = []
            vis_result = []
            tm = time.time()
            maps = np.zeros((h, w))
            for box in bbox:
                xmin, ymin, xmax, ymax , score = box
                maps[int(ymin):int(ymax), int(xmin):int(xmax)] = 1
#             for a in seg:
#                 m = MASK.decode(a)
#                 maps.append(m)
#             maps = np.sum(np.array(maps), axis=0)
#             maps[maps>=1] = 1
            tn = time.time()
            kernel = np.ones((5,5),np.uint8) 
            maps = cv2.dilate(maps,kernel,iterations = 1)
            contours, hierarchy = cv2.findContours((maps).astype(np.uint8), cv2.RETR_TREE,
                                                                 cv2.CHAIN_APPROX_SIMPLE)
            for contour in contours:
                xmin, ymin, w, h = cv2.boundingRect(contour)
                xmax = xmin+w
                ymax = ymin+h
                region_box = []
                region_mask = []
                region_vis = []
                for i, bb in enumerate(bbox):
                    if(bb[0]>=xmin and bb[1]>=ymin and bb[2]<=xmax and bb[3]<=ymax):
                        region_box.append(bbox[i])
                        region_mask.append(seg[i])
                        region_vis.append(vis[i])
                if(len(region_box) <= 0):
                    continue
                keep_ind = py_cpu_pnms(region_box, region_vis, thresh)
                bbox_result += list(np.array(region_box)[keep_ind])
                seg_result += list(np.array(region_mask)[keep_ind])
                vis_result += list(np.array(region_vis)[keep_ind])
            ann[name]['bbox'][cls] = bbox_result
            ann[name]['seg'][cls] = seg_result
            ann[name]['vis'][cls] = vis_result
            t2 = time.time()
                
    return ann
thresh=0.5
# result = copy.deepcopy(ann)
ann = mmcv.load("eval_temp.pkl")
result = nms(ann, thresh)


/home/xfr/.conda/envs/mmdet/lib/python3.6/site-packages/ipykernel_launcher.py:24: RuntimeWarning: divide by zero encountered in true_divide
/home/xfr/.conda/envs/mmdet/lib/python3.6/site-packages/ipykernel_launcher.py:24: RuntimeWarning: invalid value encountered in true_divide
/home/xfr/.conda/envs/mmdet/lib/python3.6/site-packages/ipykernel_launcher.py:25: RuntimeWarning: invalid value encountered in less_equal


In [68]:
mmcv.dump(result, "postnms.pkl")

In [75]:
from mmdet.ops.nms import nms_wrapper
def nms(ann, thresh):
    for name in tqdm(ann.keys()):
        info = ann[name]
        for cls in range(len(dataset.CLASSES)):
            bbox = np.array(info['bbox'][cls], np.float32)
            seg = np.array(info['seg'][cls])
            vis = np.array(info['vis'][cls])
            if(len(bbox)<=0):
                continue
            h, w = seg[0]['size']
            #print(bbox.shape)
            _, inds = nms_wrapper.nms(bbox, thresh)
           # print(len(inds))
            ann[name]['bbox'][cls] = bbox[inds]
            ann[name]['seg'][cls] = seg[inds]
            ann[name]['vis'][cls] = vis[inds]
    return ann
thresh=0.5
# result = copy.deepcopy(ann)
ann = mmcv.load("eval_temp.pkl")
result2 = nms(ann, thresh)

In [76]:
ann = result2
cfg = mmcv.Config.fromfile("configs/rs_test.py")
dataset = get_dataset(cfg.data.val)
for idx in tqdm(range(len(dataset))):
    img_info = dataset.img_infos[idx]
    img = cv2.imread("/home/xfr/rssid/data/val/images/"+img_info['filename'])
    filename = img_info['filename'].split(".")[0]
    info = ann[filename]
    seg = info['vis']
    for cls in range(len(dataset.CLASSES)):
        polys = seg[cls]
        for poly in polys:
            temp = np.array([poly], np.int32)
            cv2.polylines(img, temp, 1, (0, 255, 0), 1)
    cv2.imwrite("./result2/{}".format(filename)+".jpg", img)

loading annotations into memory...
Done (t=0.36s)
creating index...
index created!


In [19]:
ann = mmcv.load("./eval_temp.pkl")
cfg = mmcv.Config.fromfile("configs/rs_test.py")
dataset = get_dataset(cfg.data.val)
for idx in tqdm(range(len(dataset))):
    img_info = dataset.img_infos[idx]
    img = cv2.imread("/home/xfr/rssid/data/val/images/"+img_info['filename'])
    filename = img_info['filename'].split(".")[0]
    info = ann[filename]
    seg = info['vis']
    for cls in range(len(dataset.CLASSES)):
        polys = seg[cls]
        for poly in polys:
            temp = np.array([poly], np.int32)
            cv2.polylines(img, temp, 1, (0, 255, 0), 1)
    cv2.imwrite("./prenms/{}".format(filename)+".jpg", img)

loading annotations into memory...
Done (t=0.29s)
creating index...
index created!


In [77]:
from mmdet.core import results2json, coco_eval
outputs = []
ann = result2
cfg = mmcv.Config.fromfile("configs/rs_test.py")
dataset = get_dataset(cfg.data.val)
for idx in range(len(dataset)):
    img_info = dataset.img_infos[idx]
    filename = img_info['filename'].split(".")[0]
    if filename not in ann:
        #print(filename)
        ann[filename] = {"bbox": [np.array([]) for i in range(len(dataset.CLASSES))], "seg":[[] for i in range(len(dataset.CLASSES))]}
    info = ann[filename]
    #print(info['seg'][0])
    m = np.array(info['bbox'])
    for i in range(len(m)):
        m[i] = np.array(m[i])
    temp = (m, info['seg'])
    outputs.append(temp)
result_files = results2json(dataset, outputs, "xx")
coco_eval(result_files, ['bbox', 'segm'], dataset.coco)

loading annotations into memory...
Done (t=0.38s)
creating index...
index created!
Loading and preparing results...
DONE (t=0.47s)
creating index...
index created!
Running per image evaluation...
Evaluate annotation type *bbox*
DONE (t=167.40s).
Accumulating evaluation results...
DONE (t=0.81s).
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.344
 Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets=100 ] = 0.547
 Average Precision  (AP) @[ IoU=0.75      | area=   all | maxDets=100 ] = 0.365
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= small | maxDets=100 ] = 0.246
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=medium | maxDets=100 ] = 0.382
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= large | maxDets=100 ] = 0.489
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=  1 ] = 0.120
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets= 10 ] = 0.303
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDe